# Store Sales - Time Series Forecasting
*Use machine learning to predict grocery sales*

In [96]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(style="whitegrid", palette="muted")

RANDOM_STATE = 101


### Data collection

In [97]:
df_train = pd.read_csv(
    "./data/train.csv",
    usecols=["id", "date", "store_nbr", "family", "sales", "onpromotion"],
    dtype={
        "store_nbr": "category",
        "family": "category",
        "sales": "float32",
        "onpromotion": "uint32",
    },
    parse_dates=["date"],
)

df_train.shape

(3000888, 6)

In [98]:
df_test = pd.read_csv(
    "./data/test.csv",
    usecols=["id", "date", "store_nbr", "family", "onpromotion"],
    dtype={
        "store_nbr": "category",
        "family": "category",
        "onpromotion": "uint32",
    },
    parse_dates=["date"],
)

df_test.shape

(28512, 5)

In [99]:
pd.concat([df_train.head(2), df_train.tail(2)])

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.0,8
3000887,3000887,2017-08-15,9,SEAFOOD,16.0,0


In [100]:
pd.concat([df_test.head(2), df_test.tail(2)])

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
28510,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9
28511,3029399,2017-08-31,9,SEAFOOD,0


In [101]:
df_holidays = pd.read_csv(
    "./data/holidays_events.csv",
    # dtype={
    #     "type": "category",
    #     "locale": "category",
    # },
    parse_dates=["date"],
    # index_col="date",
)
pd.concat([df_holidays.head(2), df_holidays.tail(2)])

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
348,2017-12-25,Holiday,National,Ecuador,Navidad,False
349,2017-12-26,Additional,National,Ecuador,Navidad+1,False


In [102]:
df_stores = pd.read_csv(
    "./data/stores.csv",
    dtype={
        "store_nbr": "category",
        "city": "category",
        "state": "category",
        "type": "category",
        "cluster": "category",
    },
)

pd.concat([df_stores.head(2), df_stores.tail(2)])

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
52,53,Manta,Manabi,D,13
53,54,El Carmen,Manabi,C,3


In [103]:
df_oil = pd.read_csv(
    "./data/oil.csv",
    parse_dates=["date"],
    # index_col="date",
)

pd.concat([df_oil.head(2), df_oil.tail(2)])

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
1216,2017-08-30,45.96
1217,2017-08-31,47.26


In [104]:
df_txns = pd.read_csv(
    "./data/transactions.csv",
    dtype={
        "store_nbr": "category",
        "transactions": "uint32",
    },
    parse_dates=["date"],
    # index_col="date",
)

pd.concat([df_txns.head(2), df_txns.tail(2)])

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
83486,2017-08-15,53,932
83487,2017-08-15,54,802


### EDA

#### Check nulls

In [105]:
def show_missing_data(df, df_name):
    """
    Display number and percentage of columns with any missing value
    """
    total = df.isnull().sum().sort_values(ascending=False)
    percent = (
        ((df.isnull().sum() / df.isnull().count()) * 100)
        .sort_values(ascending=False)
        .round(2)
    )
    missing_data = pd.concat([total, percent], axis=1, keys=["# missing", "% missing"])
    
    if missing_data["# missing"].max() > 0:
        print(f'`{df_name}` has null values:')
        print(missing_data[missing_data["# missing"] > 0])
    else:
        print(f'`{df_name}` does not have null values')

In [106]:
show_missing_data(df_train, 'df_train')
show_missing_data(df_test, 'df_test')
show_missing_data(df_txns, 'df_txns')
show_missing_data(df_holidays, 'df_holidays')
show_missing_data(df_oil, 'df_oil')

`df_train` does not have null values
`df_test` does not have null values
`df_txns` does not have null values
`df_holidays` does not have null values
`df_oil` has null values:
            # missing  % missing
dcoilwtico         43       3.53


### Data cleaning & Preprocessing

#### Manage nulls

In [107]:
# Careful!! there can be 3 days in a row without values.
# -> TO BE UPDATED TO FIND ITERATIVELLY THE NEAREST VALUE.

def fill_missing_with_surrounding_mean(df, target_column, min_periods=2, decimals=2):
    """
    fill null values on field `target_column` with their surrounding `min_periods` peers
    """
    # Calculate the rolling mean with the specified minimum number of periods
    rolling_mean = (
        df[target_column]
        .rolling(window=min_periods * 2 + 1, min_periods=min_periods, center=True)
        .mean()
    )

    # Round the rolling mean to the specified number of decimal places
    rounded_mean = rolling_mean.round(decimals)

    # Fill missing values in the target column with the rolling mean
    df[target_column].fillna(rounded_mean, inplace=True)

    return df

# Alternatively:
# df_oil['dcoilwtico'].fillna(method='backfill', inplace=True)

### Process dataset

In [108]:
# df_oil = fill_missing_with_surrounding_mean(df_oil, 'dcoilwtico', 1)
# show_missing_data(df_oil, 'df_oil')

### EDA

In [109]:
# analyses will be done at (entire) day level, where specific time is not relevant,
# so converting to Period object

# but... according to gpt: in the context of time series modeling, especially when 
# using functions like DeterministicProcess from statsmodels, it's generally more 
# straightforward to keep dates as a DateTime type rather than converting them to
#  a Period. The DateTimeIndex in pandas is more widely supported for time series 
# analysis and makes certain operations more straightforward.

# df_train['date'] = df_train.date.dt.to_period('D')
# df_test['date'] = df_test.date.dt.to_period('D')
# df_holidays['date'] = df_holidays.date.dt.to_period('D')
# df_oil['date'] = df_oil.date.dt.to_period('D')
# df_txns['date'] = df_txns.date.dt.to_period('D')


In [110]:
# sns.histplot(np.log1p(df_train['sales']), kde=True)

In [111]:
# df_train['sales'].plot(figsize=(10, 6))
# plt.title('Sales over Time')
# plt.show()

In [112]:
# plt.figure(figsize=(5,3))
# sns.lineplot(data=df_oil, x='date', y='dcoilwtico')

### Feature engineering

In [113]:
def lag_features(df, lags):
    for lag in lags:
        # Create column 'sales_t-lag' by taking previous values ​​of column 'sales' based on columns 'store_nbr' and 'family'
        df[f"sales_t-{lag}"] = df.groupby(["store_nbr", "family"])["sales"].transform(
            lambda x: x.shift(lag)
        )
    return df

In [114]:
def fill_na(df):
    df["holiday_type"] = df["holiday_type"].fillna("Common")
    df["locale"] = df["locale"].fillna("Common")
    df["description"] = df["description"].fillna("Unknown")
    df["transferred"] = df["transferred"].fillna(False)
    # TODO: to be replaced by fill_missing_with_surrounding_mean()
    df["dcoilwtico"] = df["dcoilwtico"].fillna(method="backfill")
    df["transactions"] = df["transactions"].fillna(0)

    return df

In [115]:
def merge_data(df):
    df = (
        df.merge(df_stores, left_on="store_nbr", right_on="store_nbr", how="left")
        .rename(columns={"type": "store_type"})
        .merge(
            df_txns,
            left_on=["date", "store_nbr"],
            right_on=["date", "store_nbr"],
            how="left",
        )
        .merge(df_holidays, left_on="date", right_on="date", how="left")
        .drop_duplicates(subset="id")
        .rename(columns={"type": "holiday_type"})
        .merge(df_oil, left_on="date", right_on="date", how="left")
    )
    return df.drop('id', axis=1)

In [116]:
def create_date_features(df):
    df["month"] = df["date"].dt.month
    df["day_of_month"] = df["date"].dt.day
    df["day_of_year"] = df["date"].dt.dayofyear
    df["week_of_year"] = df["date"].dt.isocalendar().week
    df["day_of_week"] = df["date"].dt.dayofweek
    df["year"] = df["date"].dt.year
    return df

In [117]:
def random_noise(dataframe):
    """
    Generate random noise with a normal distribution (mean=0, std=2)
    matching the length of the dataframe.
    """
    return np.random.normal(scale=2.0, size=(len(dataframe),))

In [118]:
def roll_mean_features(dataframe, windows):
    """
    Calculate rolling mean with triangular window and custom
    noise for 'sales' within 'store_nbr' and 'family' groups.
    """
    def add_noise(x):
        return x + np.random.normal(size=len(x))

    for window in windows:
        dataframe["sales_roll_mean_" + str(window)] = dataframe.groupby(
            ["store_nbr", "family"]
        )["sales"].transform(
            lambda x: x.shift(16)
            .rolling(window=window, min_periods=7, win_type="triang")
            .mean()
        )
        dataframe["sales_roll_mean_" + str(window)] = dataframe.groupby(
            ["store_nbr", "family"]
        )["sales_roll_mean_" + str(window)].transform(add_noise)
    return dataframe

In [119]:
def ewma_features(dataframe, alphas, lags):
    """
    Create Exponentially Weighted Moving Average (EWMA) features for 'sales'
    with specified alphas and lags, grouped by 'store_nbr' and 'family'.
    """
    for alpha in alphas:
        for lag in lags:
            feature_name = (
                "sales_ewm_alpha_" + str(alpha).replace(".", "") + "_lag_" + str(lag)
            )
            dataframe[feature_name] = dataframe.groupby(["store_nbr", "family"])[
                "sales"
            ].transform(lambda x: x.shift(lag).ewm(alpha=alpha, min_periods=1).mean())
    return dataframe

In [120]:
# choose relevant features for regression models
col = [
    "date",
    "store_nbr",
    "family",
    "sales",
    "onpromotion",
    "cluster",
    "holiday_type",
    "locale",
    # "description",
    "transferred",
    "dcoilwtico",
]

In [121]:
df_train['sales'] = np.log1p(df_train['sales'])

df_both = pd.concat([df_train, df_test], axis=0)
df_both = merge_data(df_both)
df_both = fill_na(df_both)
df_both = create_date_features(df_both)
# df_both = lag_features(
#     df_both, lags=[*range(1, 16), 16, 17, 18, 19, 20, 21, 22, 30, 31, 90, 180, 364]
# )
# df_both = roll_mean_features(df_both, [16, 17, 18, 30])
df_both = df_both[col]

/var/folders/1d/ty_knwmj61b4d3qs091tfglm0000gn/T/ipykernel_15670/1010387790.py:7: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df["dcoilwtico"] = df["dcoilwtico"].fillna(method="backfill")


In [122]:
# alphas = [0.95, 0.9, 0.8, 0.5]
# lags =[1, 7,30]
# all_df = ewma_features(df_all, alphas, lags)

In [123]:
# df_all["store_nbr"] = df_all["store_nbr"].astype("category")
# df_all["family"] = df_all["family"].astype("category")
# df_all["store_nbr"] = df_all["store_nbr"].astype("category")
# df_all["cluster"] = df_all["cluster"].astype("category")
# df_all["family"] = df_all["family"].astype("category")
# df_all["holiday_type"] = df_all["holiday_type"].astype("category")
# df_all["locale"] = df_all["locale"].astype("category")
# df_all["description"] = df_all["description"].astype("category")

In [124]:
# df_both = df_both[df_both['date'] > '2013-12-31']
# X = df_both[df_both['date'] <= '2017-08-15'].drop('date', axis=1)
# X_test = df_both[df_both['date'] > '2017-08-15'].drop('date', axis=1)
# corr = X.select_dtypes(include=['number']).corr()
# corr["sales"].sort_values(ascending=False)
# y = X['sales']
# X = X.drop(['sales'], axis=1)
# X_test = X_test.drop(['sales'], axis=1)


In [125]:
# TBC: start from 2014 to have the preceeding Y-1 ?
df_both_filtered = df_both[df_both['date'] > '2013-12-31']

### Encoding categorical data

In [126]:
def encode_features(df):
    """Encode categorical features"""
    columns_to_encode = df.select_dtypes(
        include=["object", "category"]
    ).columns.tolist()
    encoded_df_train = pd.get_dummies(df, columns=columns_to_encode, drop_first=True)

    # replace spaces in df columns for lightxgb model
    encoded_df_train.columns = encoded_df_train.columns.str.replace(" ", "_")

    return encoded_df_train

In [127]:
df_both_encoded = encode_features(df_both_filtered)

In [128]:
# X = df_all_encoded[df_all_encoded["date"] <= "2017-08-15"].drop(
#     ["sales", "date"], axis=1
# )
# y = df_all_encoded[df_all_encoded["date"] <= "2017-08-15"]["sales"]

df_train_encoded = df_both_encoded[df_both_encoded["date"] <= "2017-08-15"]
df_test_encoded = df_both_encoded[df_both_encoded["date"] > "2017-08-15"]

In [129]:
# from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.2, random_state=RANDOM_STATE
# )

### Model v2

In [130]:
def rmsle(y_true_log, y_pred_log):
    # """
    # Compute Root Mean Squared Logarithmic Error for log-transformed targets.
    # Parameters:
    # - y_true_log: Log-transformed actual values
    # - y_pred_log: Log-transformed predicted values
    # """
    return np.sqrt(mean_squared_error(y_true_log, y_pred_log))

### Lasso & Ridge

In [131]:
# convert type in field `date` from datetime64[ns] to period[D]
df_train['date'] = df_train.date.dt.to_period('D')

In [132]:
df_train.drop('id', axis=1, inplace=True)

In [133]:
df_train = df_train.set_index(["store_nbr", "family", "date"]).sort_index()

In [134]:
y = df_train.unstack(['store_nbr', 'family']).loc["2017"]
y.head(5)

sales                                                       \
store_nbr           1                                                        
family     AUTOMOTIVE BABY CARE    BEAUTY BEVERAGES     BOOKS BREAD/BAKERY   
date                                                                         
2017-01-01   0.000000       0.0  0.000000  0.000000  0.000000     0.000000   
2017-01-02   1.791759       0.0  0.000000  7.268920  0.000000     5.122886   
2017-01-03   1.609438       0.0  1.609438  8.033334  1.098612     6.254498   
2017-01-04   0.693147       0.0  1.609438  8.019613  1.098612     6.299410   
2017-01-05   1.098612       0.0  1.386294  7.870166  0.000000     6.281294   

                                                      ... onpromotion        \
store_nbr                                             ...           9         
family     CELEBRATION  CLEANING     DAIRY      DELI  ...   MAGAZINES MEATS   
date                                                  ...                     
2017-01-01    0.000000  0.000000  0.000000  0.000000  ...           0     0   
2017-01-02    0.000000  5.808143  5.932245  3.828207  ...           0     0   
2017-01-03    2.772589  6.859615  6.952729  5.348535  ...           0     0   
2017-01-04    2.890372  6.962244  6.937314  4.919572  ...           0     0   
2017-01-05    3.713572  6.823286  6.749931  4.927297  ...           0    21   

                                                                       \
store_nbr                                                               
family     PERSONAL CARE PET SUPPLIES PLAYERS AND ELECTRONICS POULTRY   
date                                                                    
2017-01-01             0            0                       0       0   
2017-01-02            13            0                       0       2   
2017-01-03            11            0                       0       1   
2017-01-04            15            0                       0       1   
2017-01-05             8            0                       0       1   

                                                                      
store_nbr                                                             
family     PREPARED FOODS PRODUCE SCHOOL AND OFFICE SUPPLIES SEAFOOD  
date                                                                  
2017-01-01              0       0                          0       0  
2017-01-02              1       4                          0       0  
2017-01-03              2     150                          0       0  
2017-01-04              8       9                          0       0  
2017-01-05              1       5                          0       1  

[5 rows x 3564 columns]

In [135]:
y.info()

<class 'pandas.core.frame.DataFrame'>
PeriodIndex: 227 entries, 2017-01-01 to 2017-08-15
Freq: D
Columns: 3564 entries, ('sales', '1', 'AUTOMOTIVE') to ('onpromotion', '9', 'SEAFOOD')
dtypes: float32(1782), uint32(1782)
memory usage: 3.1 MB


In [136]:
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess

fourier = CalendarFourier(freq='M', order=3)

dp = DeterministicProcess(
    index=y.index,
    constant=False,
    order=5,
    seasonal=True,
    additional_terms=[fourier],
    drop=True,
)

X = dp.in_sample()
X['NewYear'] = (X.index.dayofyear == 1)

In [137]:
X.head(3)

,trend,trend_squared,trend_cubed,trend**4,trend**5,"s(1,7)","s(2,7)","s(3,7)","s(4,7)","s(5,7)","s(6,7)","s(7,7)","sin(1,freq=M)","cos(1,freq=M)","sin(2,freq=M)","cos(2,freq=M)","sin(3,freq=M)","cos(3,freq=M)",NewYear
date,,,,,,,,,,,,,,,,,,,
2017-01-01,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,True
2017-01-02,2.0,4.0,8.0,16.0,32.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.201299,0.979530,0.394356,0.918958,0.571268,0.820763,False
2017-01-03,3.0,9.0,27.0,81.0,243.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.394356,0.918958,0.724793,0.688967,0.937752,0.347305,False


In [138]:
from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.001, random_state=13
# )
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=RANDOM_STATE
)

from sklearn.preprocessing import RobustScaler

transformerL = RobustScaler().fit(X_train)

X_test = transformerL.transform(X_test)
X_train = transformerL.transform(X_train)

In [139]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

def evaluate_models(regressors, X, y):
    results = []
    for regressor in regressors:
        y_pred = regressor.predict(X)
        # TODO: shouldn't happen, but...
        y_pred[y_pred < 0] = 0
        mae = mean_absolute_error(y, y_pred)
        rmse = np.sqrt(mean_squared_error(y, y_pred))
        results.append(
            {
                "Model": type(regressor).__name__,
                "MAE": mae,
                "RMSE": rmse,
            }
        )
    sorted_results = sorted(results, key=lambda x: x["RMSE"], reverse=False)
    print(
        f"{'Model':<25} {'RMSE':<11} {'MAE':<8}"
    )
    print("-" * 72)
    for result in sorted_results:
        metrics = f"{result['RMSE']:<11.5f} {result['MAE']:<8.2f}"
        print(f"{result['Model']:<25} {metrics}")

In [140]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn import metrics

lasso = Lasso(alpha=1, fit_intercept=True, max_iter=7000).fit(X_train, y_train)

ridge = Ridge(alpha=0.4, fit_intercept=True, max_iter=7000).fit(X_train, y_train)


/Users/wint3rmute/Gro/datascilabs/vtensorflow/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 0.000e+00, tolerance: 0.000e+00
  model = cd_fast.enet_coordinate_descent(


In [141]:
models = [ridge, lasso]
evaluate_models(models, X_test, y_test)

Model                     RMSE        MAE     
------------------------------------------------------------------------
Ridge                     3.23429     1.06    
Lasso                     3.67253     1.34    


In [142]:
"""
Based on your description, df_test has 28,512 records, spanning dates from 2017-08-16 to 2017-08-31,
with approximately 1,782 records per day. This structure suggests that for each day, there are 1,782
combinations of 'store_nbr' and 'family' products. Given that there are 16 days in your test date range,
this aligns with the total record count (16 days * 1,782 records/day = 28,512 records).
"""

"\nBased on your description, df_test has 28,512 records, spanning dates from 2017-08-16 to 2017-08-31,\nwith approximately 1,782 records per day. This structure suggests that for each day, there are 1,782\ncombinations of 'store_nbr' and 'family' products. Given that there are 16 days in your test date range,\nthis aligns with the total record count (16 days * 1,782 records/day = 28,512 records).\n"

In [143]:
import copy

df_test['date'] = df_test.date.dt.to_period('D')
df_test = df_test.set_index(['store_nbr', 'family', 'date']).sort_index()

X_test = dp.out_of_sample(steps=16)
X_test.index.name = 'date'
X_test['NewYear'] = (X_test.index.dayofyear == 1)

X_test_copy = copy.deepcopy(X_test)

X_test = transformerL.transform(X_test)

predictionL = lasso.predict(X_test)
predictionL[predictionL < 0] = 0



In [144]:
y_submit = pd.DataFrame(np.expm1(predictionL), index=X_test_copy.index, columns=y.columns)
# y_submit = pd.DataFrame(predictionL, index=X_test_copy.index, columns=y.columns)
y_submit = y_submit.stack(['store_nbr', 'family'])
y_submit = y_submit.join(df_test.id).reindex(columns=['id', 'sales'])

In [145]:
y_submit

id        sales
date       store_nbr family                                          
2017-08-16 1         AUTOMOTIVE                  3000888     3.283707
                     BABY CARE                   3000889     0.000000
                     BEAUTY                      3000890     2.689846
                     BEVERAGES                   3000891  2004.868932
                     BOOKS                       3000892     0.320290
...                                                  ...          ...
2017-08-31 9         POULTRY                     3029395   418.607620
                     PREPARED FOODS              3029396   117.379339
                     PRODUCE                     3029397  1579.561665
                     SCHOOL AND OFFICE SUPPLIES  3029398     3.194469
                     SEAFOOD                     3029399    16.123541

[28512 rows x 2 columns]

In [146]:
# y_submit.to_csv('submission.csv', index=False)

In [147]:
# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Dropout, BatchNormalization

# X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=13)

# from sklearn.preprocessing import RobustScaler

# transformerKERAS = RobustScaler().fit(X_train)

# X_val = transformerKERAS.transform(X_val)
# X_train = transformerKERAS.transform(X_train)

# from tensorflow.keras.callbacks import ReduceLROnPlateau
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=20, min_lr=0.000001, verbose=1, mode='min')

# def create_model():

#     model = Sequential()
#     model.add(Dense(units=500, activation='relu', input_dim=19))
#     model.add(Dense(units=2000, activation='relu'))
#     model.add(Dense(units=1500, activation='relu'))
#     model.add(Dense(units=1782, activation='swish'))

#     model.compile(loss='mae', optimizer='adam')
    
#     return model

# model = create_model()
# model.fit(X_train, y_train, epochs=5000, batch_size=2000, validation_data=(X_val, y_val),callbacks=[reduce_lr])

# y_pred1 = model.predict(X_val)
# #print(model1.score(X_val, y_val))

# y_pred1[y_pred1 < 0] = 0

# print('Mean Absolute Error:', metrics.mean_absolute_error(y_val, y_pred1))
# print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_val, y_pred1)))

# print('======')